# MAPT Haplotype Analyses
* The 6-variant-defiened haplotypes are:
* **rs1467967, rs242557, rs3785883, rs2471738, rs8070723, rs7521**
* **rs8070723** is the Tagging Variant where
   * The Minor allele of **rs8070723** corresponds to **MAPT H2** haplotype and 
   * The Major allele of **rs8070723** corresponds to the **MAPT H1** haplotype
* Five other MAPT common variants describe the subhaplotype structure and include:
   * **rs1467967, rs242557** -> the H1c haplotype-tagging variant
   * **rs3785883, rs2471738, and rs8070723** which along with **rs7521**  define the H1 subhaplotypes
     

In [2]:
## Libraries
library(haplo.stats)
library(ggplot2)
library(tidyverse)
seed <- c(17, 53, 1, 40, 37, 0, 62, 56, 5, 52, 12, 1)
set.seed(seed)

In [3]:
packageVersion('haplo.stats')

[1] ‘1.9.7’

In [4]:
R.version.string

[1] "R version 4.3.3 (2024-02-29)"

In [60]:
## Dirs
data_dir='path/to/data'

## Filenames
ped_file='filename.ped'
map_file='filename.map'


## Data

In [15]:
### DATA MAPT .ped & .map files
## MAP file
map_data <- read.table(paste0(data_dir, map_file), header = FALSE)
## PED File (Genotype Data)
ped_data <- read.table(paste0(data_dir, ped_file), header = FALSE)
## PCA file
#

# Extract the genotype data (from the 7th column onwards)
geno_data <- ped_data[, 7:ncol(ped_data)]

# The Phenotype (case/control phenotype, binary: 0 = control, 1 = case) is in the 6th column
phenotype <- ped_data[, 6]
# Recode phenotype: 1 -> 0 (Control), 2 -> 1 (Case)
phenotype <- ifelse(phenotype == 1, 0, 1)
# The sex (covariate) is in the 5th column
sex <- ped_data[, 5]


In [16]:
head(map_data)

,V1,V2,V3,V4
,<int>,<chr>,<int>,<int>
1,17,rs1467967,0,45908813
2,17,rs242557,0,45942346
3,17,rs3785883,0,45977067
4,17,rs2471738,0,45998697
5,17,rs8070723,0,46003698
6,17,rs7521,0,46028029


In [17]:
head(geno_data, 2)

,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,G,G,A,G,G,G,C,C,A,A,G,G
2,G,A,G,G,A,G,C,C,A,A,A,A


In [18]:
cat('outcome',phenotype[1:20], '\n')
cat('covariate', sex[1:20] )

outcome 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 
covariate 1 2 2 2 1 2 1 1 1 1 2 1 1 1 1 1 1 1 2 2

# Exploratory analysis

## Create a Genotype Matrix

In [19]:
## Assign labels to geno data
# Create new column names by appending ".a1" and ".a2" for each SNP

## Add labels to geno matrix
snp_names= map_data$V2
cat('labels :', snp_names)

new_column_names <- c()
for (snp in snp_names) {
  new_column_names <- c(new_column_names, paste0(snp, ".a1"), paste0(snp, ".a2"))
}
# Assign the new column names to the data frame
colnames(geno_data) <- new_column_names


labels : rs1467967 rs242557 rs3785883 rs2471738 rs8070723 rs7521

In [20]:
head(geno_data,4)

,rs1467967.a1,rs1467967.a2,rs242557.a1,rs242557.a2,rs3785883.a1,rs3785883.a2,rs2471738.a1,rs2471738.a2,rs8070723.a1,rs8070723.a2,rs7521.a1,rs7521.a2
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,G,G,A,G,G,G,C,C,A,A,G,G
2,G,A,G,G,A,G,C,C,A,A,A,A
3,G,G,A,A,G,G,C,C,A,A,G,A
4,G,A,A,A,G,G,C,C,A,A,G,A


## 2. Estimate Haplotype Frequency with `haplo.em`

In [22]:
save.em <- haplo.em(geno=geno_data, locus.label=snp_names, miss.val=c(0,NA))
names(save.em)

[1] "lnlike"        "lnlike.noLD"   "lr"            "df.lr"        
 [5] "hap.prob"      "hap.prob.noLD" "converge"      "locus.label"  
 [9] "indx.subj"     "subj.id"       "post"          "hap1code"     
[13] "hap2code"      "haplotype"     "nreps"         "rows.rem"     
[17] "max.pairs"     "control"

In [24]:
print(save.em)

                                   Haplotypes                                    
   rs1467967 rs242557 rs3785883 rs2471738 rs8070723 rs7521 hap.freq
1          A        A         A         C         A      A  0.02704
2          A        A         A         C         A      G  0.00239
3          A        A         A         T         A      A  0.00065
4          A        A         A         T         A      G  0.00315
5          A        A         G         C         A      A  0.03761
6          A        A         G         C         A      G  0.04990
7          A        A         G         T         A      A  0.00166
8          A        A         G         T         A      G  0.04603
9          A        G         A         C         A      A  0.17720
10         A        G         A         C         A      G  0.01621
11         A        G         A         T         A      A  0.00049
12         A        G         A         T         A      G  0.01719
13         A        G         G   

In [25]:
dim(save.em)

NULL

## Remark

* The print methods shows the haplotypes and their estimated frequencies, followed by the final log-likelihood
statistic and the lr stat for no LD, which is the likelihood ratio test statistic contrasting the lnlike for the
estimated haplotype frequencies versus the lnlike under the null assuming that alleles from all loci are in
linkage equilibrium.

### Summary method

In [27]:
summary(save.em, nlines=7)

             Subjects: Haplotype Codes and Posterior Probabilities               
  subj.id hap1 hap2 posterior
1       1   23   31   1.00000
2       2    9   30   0.85431
3       2   13   26   0.14569
4       3   22   23   1.00000
5       4    6   22   0.69726
6       4    5   23   0.30274
7       5   22   29   0.35464
                     Number of haplotype pairs: max vs used                      
    
x       1    2    4    8   16
  1  3521    0    0    0    0
  2   206 3072    0    0    0
  4   534    0 2300    0    0
  8   232    0    0 1103    0
  16   31    0    0    0  170
  32    1    0    0    0    0


### Remark
* The first part of the summary output lists the subject id (row number of input geno matrix), the codes for the haplotypes of each pair, and the posterior probabilities of the haplotype pairs.
* The second part gives a table of the maximum number of pairs of haplotypes per subject, versus the number of pairs used
in the final posterior probabilities. 
*  The haplotype codes remove the clutter of illustrating all the alleles of the haplotypes, but may not be as informative as the actual haplotypes themselves.
* To see the actual haplotypes, use the show.haplo=TRUE option, as in the following example.

In [28]:
# show full haplotypes, instead of codes
summary(save.em, show.haplo=TRUE, nlines=7)

             Subjects: Haplotype Codes and Posterior Probabilities               
  subj.id hap1.rs1467967 hap1.rs242557 hap1.rs3785883 hap1.rs2471738
1       1              G             A              G              C
2       2              A             G              A              C
3       2              A             G              G              C
4       3              G             A              G              C
5       4              A             A              G              C
6       4              A             A              G              C
7       5              G             A              G              C
  hap1.rs8070723 hap1.rs7521 hap2.rs1467967 hap2.rs242557 hap2.rs3785883
1              A           G              G             G              G
2              A           A              G             G              G
3              A           A              G             G              A
4              A           A              G             A              G
5

## Haplotype Frequencies by Group Subsets using `haplo.group`

In [29]:
#help(haplo.group)

In [31]:
## Frequencies
group.bin=haplo.group(phenotype, geno_data, locus.label=snp_names, miss.val=0)
print(group.bin, nlines=15)


-------------------------------------------------------------------------------- 
                       Counts per Grouping Variable Value                        
-------------------------------------------------------------------------------- 
group
   0    1 
6364 4806 


-------------------------------------------------------------------------------- 
                        Haplotype Frequencies By Group                           
-------------------------------------------------------------------------------- 
   rs1467967 rs242557 rs3785883 rs2471738 rs8070723 rs7521   Total phenotype.0
1          A        A         A         C         A      A 0.02704     0.02725
2          A        A         A         C         A      G 0.00239     0.00217
3          A        A         A         T         A      A 0.00065     0.00026
4          A        A         A         T         A      G 0.00315     0.00346
5          A        A         G         C         A      A 0.03761     0.03712
6  

### Remark
* The group.bin object can be very large, depending on the number of possible haplotypes, so only a portion of the output is illustrated above (limited again by nlines). The first section gives a short summary of how many subjects appear in each of the groups. The second section is a table with the following columns:
    *  The first column gives row numbers.
    *  Total are the estimated haplotype frequencies for the entire data set.
    *  The last columns are the estimated haplotype frequencies for the subjects in the levels of the group variable (phenotye.0 and phenotype.1).
    *  Note that some haplotype frequencies have an NA, which appears when the haplotypes do not occur in the subgroups

In [314]:
class(group.bin)

[1] "haplo.group" "data.frame"

In [32]:
head(group.bin$group.df,3)

,rs1467967,rs242557,rs3785883,rs2471738,rs8070723,rs7521,Total,phenotype=0,phenotype=1
,<I<chr>>,<I<chr>>,<I<chr>>,<I<chr>>,<I<chr>>,<I<chr>>,<dbl>,<dbl>,<dbl>
1,A,A,A,C,A,A,0.0270367931,0.027249175,0.026698908
2,A,A,A,C,A,G,0.0023883317,0.002166285,0.002835009
3,A,A,A,T,A,A,0.0006465266,0.000262420,0.001381547


In [33]:
cat('Number of controls and cases', group.bin$group.count,'\n')
cat('Number of MAPT loci: ', group.bin$n.loci, '\n' )
freq_haplotypes= group.bin$group.df
cat('Total number of MAPT haplotypes', dim(freq_haplotypes), '\n')

Number of controls and cases 6364 4806 
Number of MAPT loci:  6 
Total number of MAPT haplotypes 34 9 


In [34]:
group.bin$group.count

group
   0    1 
6364 4806 

**Note:**
* A total of 34 haplotypes created from the 6 MAPT variants

In [35]:
summary(freq_haplotypes$Total)

    Min.  1st Qu.   Median     Mean  3rd Qu.     Max.     NA's 
0.000000 0.002388 0.011283 0.030303 0.046034 0.177200        1 

In [36]:
freq_haplotypes[freq_haplotypes$Total > 0.01,]

,rs1467967,rs242557,rs3785883,rs2471738,rs8070723,rs7521,Total,phenotype=0,phenotype=1
,<I<chr>>,<I<chr>>,<I<chr>>,<I<chr>>,<I<chr>>,<I<chr>>,<dbl>,<dbl>,<dbl>
1,A,A,A,C,A,A,0.02703679,0.02724918,0.02669891
5,A,A,G,C,A,A,0.03761375,0.03712177,0.03831520
6,A,A,G,C,A,G,0.04990283,0.05085730,0.04844548
8,A,A,G,T,A,G,0.04603415,0.04158320,0.05225495
9,A,G,A,C,A,A,0.17719945,0.17402660,0.18112497
10,A,G,A,C,A,G,0.01620877,0.01599462,0.01635189
12,A,G,A,T,A,G,0.01719111,0.01646962,0.01791642
13,A,G,G,C,A,A,0.07687231,0.07548827,0.07886540
14,A,G,G,C,A,G,0.02988668,0.03057126,0.02909069


In [37]:
dim(freq_haplotypes)

[1] 34  9

## Regression Models: `haplo.glm`

### Preparing the data.frame for haplo.glm
* A data.frame must be defined, and this object must contain the trait and other optional covariates, plus a special kind of genotype matrix (`geno.glm

* Below we prepare a genotype matrix, `geno.glm`, and create a data.frame object, `glm.data`, for use in haplo.glm.

In [38]:
# Set up data for haplo.glm, include geno.glm,
# covariates sex and respnose is phenotype (PD)

In [40]:
# Set up data for haplo.glm, include geno.glm,
# covariates age and male, and responses resp and y.bin
geno.glm <- setupGeno(geno_data,  miss.val=c(0,NA), locus.label=snp_names)
attributes(geno.glm)

$dim
[1] 11170    12

$dimnames
$dimnames[[1]]
NULL

$dimnames[[2]]
 [1] "rs1467967.a1" "rs1467967.a2" "rs242557.a1"  "rs242557.a2"  "rs3785883.a1"
 [6] "rs3785883.a2" "rs2471738.a1" "rs2471738.a2" "rs8070723.a1" "rs8070723.a2"
[11] "rs7521.a1"    "rs7521.a2"   


$class
[1] "model.matrix"

$unique.alleles
$unique.alleles[[1]]
[1] "A" "G"

$unique.alleles[[2]]
[1] "A" "G"

$unique.alleles[[3]]
[1] "A" "G"

$unique.alleles[[4]]
[1] "C" "T"

$unique.alleles[[5]]
[1] "A" "G"

$unique.alleles[[6]]
[1] "A" "G"

In [41]:
# Combine Genotype data, covariates abd respose for model fit Model fit
glm.data <- data.frame(geno.glm, sex=sex, pd_case=phenotype) # combine geno + pheno + covariates for _glm_ analysis
length(phenotype)
attributes(glm.data)

[1] 11170

$names
[1] "geno.glm" "sex"      "pd_case" 

$class
[1] "data.frame"

$row.names
    [1]     1     2     3     4     5     6     7     8     9    10    11    12
   [13]    13    14    15    16    17    18    19    20    21    22    23    24
   [25]    25    26    27    28    29    30    31    32    33    34    35    36
   [37]    37    38    39    40    41    42    43    44    45    46    47    48
   [49]    49    50    51    52    53    54    55    56    57    58    59    60
   [61]    61    62    63    64    65    66    67    68    69    70    71    72
   [73]    73    74    75    76    77    78    79    80    81    82    83    84
   [85]    85    86    87    88    89    90    91    92    93    94    95    96
   [97]    97    98    99   100   101   102   103   104   105   106   107   108
  [109]   109   110   111   112   113   114   115   116   117   118   119   120
  [121]   121   122   123   124   125   126   127   128   129   130   131   132
  [133]   133   134   135   136   137   138   139   140   141   142   143   144
  [145]   145   146   147   148   149   150   151   152   153   154   155   156
  [157]   157   158   159   160   161   162   163   164   165   166   167   168
  [169]   169   170   171   172   173   174   175   176   177   178   179   180
  [181]   181   182   183   184   185   186   187   188   189   190   191   192
  [193]   193   194   195   196   197   198   199   200   201   202   203   204
  [205]   205   206   207   208   209   210   211   212   213   214   215   216
  [217]   217   218   219   220   221   222   223   224   225   226   227   228
  [229]   229   230   231   232   233   234   235   236   237   238   239   240
  [241]   241   242   243   244   245   246   247   248   249   250   251   252
  [253]   253   254   255   256   257   258   259   260   261   262   263   264
  [265]   265   266   267   268   269   270   271   272   273   274   275   276
  [277]   277   278   279   280   281   282   283   284   285   286   287   288
  [289]   289   290   291   292   293   294   295   296   297   298   299   300
  [301]   301   302   303   304   305   306   307   308   309   310   311   312
  [313]   313   314   315   316   317   318   319   320   321   322   323   324
  [325]   325   326   327   328   329   330   331   332   333   334   335   336
  [337]   337   338   339   340   341   342   343   344   345   346   347   348
  [349]   349   350   351   352   353   354   355   356   357   358   359   360
  [361]   361   362   363   364   365   366   367   368   369   370   371   372
  [373]   373   374   375   376   377   378   379   380   381   382   383   384
  [385]   385   386   387   388   389   390   391   392   393   394   395   396
  [397]   397   398   399   400   401   402   403   404   405   406   407   408
  [409]   409   410   411   412   413   414   415   416   417   418   419   420
  [421]   421   422   423   424   425   426   427   428   429   430   431   432
  [433]   433   434   435   436   437   438   439   440   441   442   443   444
  [445]   445   446   447   448   449   450   451   452   453   454   455   456
  [457]   457   458   459   460   461   462   463   464   465   466   467   468
  [469]   469   470   471   472   473   474   475   476   477   478   479   480
  [481]   481   482   483   484   485   486   487   488   489   490   491   492
  [493]   493   494   495   496   497   498   499   500   501   502   503   504
  [505]   505   506   507   508   509   510   511   512   513   514   515   516
  [517]   517   518   519   520   521   522   523   524   525   526   527   528
  [529]   529   530   531   532   533   534   535   536   537   538   539   540
  [541]   541   542   543   544   545   546   547   548   549   550   551   552
  [553]   553   554   555   556   557   558   559   560   561   562   563   564
  [565]   565   566   567   568   569   570   571   572   573   574   575   576
  [577]   577   578   579   580   581   582   583   584   585   586   587   588

In [42]:
recoded_geno <- setupGeno(geno_data, locus.label = snp_names)
head(recoded_geno, 5)

rs1467967.a1,rs1467967.a2,rs242557.a1,rs242557.a2,rs3785883.a1,rs3785883.a2,rs2471738.a1,rs2471738.a2,rs8070723.a1,rs8070723.a2,rs7521.a1,rs7521.a2
2,2,1,2,2,2,1,1,1,1,2,2
2,1,2,2,1,2,1,1,1,1,1,1
2,2,1,1,2,2,1,1,1,1,2,1
2,1,1,1,2,2,1,1,1,1,2,1
2,2,1,2,1,2,2,1,1,1,2,1


In [749]:
head(geno_data, 2)

,rs1467967.a1,rs1467967.a2,rs242557.a1,rs242557.a2,rs3785883.a1,rs3785883.a2,rs2471738.a1,rs2471738.a2,rs8070723.a1,rs8070723.a2,rs7521.a1,rs7521.a2
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,G,G,A,G,G,G,C,C,A,A,G,G
2,G,A,G,G,A,G,C,C,A,A,A,A


# Haplotype association analysis with PD (logistic regression)
* Model is: fit.pd <- glm(casepd_ ~ sex + hapN, data = data, family =”binomial”)
* Covariates: `sex`

In [43]:
print(dim(geno.glm))
head(geno.glm, 3)

[1] 11170    12


rs1467967.a1,rs1467967.a2,rs242557.a1,rs242557.a2,rs3785883.a1,rs3785883.a2,rs2471738.a1,rs2471738.a2,rs8070723.a1,rs8070723.a2,rs7521.a1,rs7521.a2
2,2,1,2,2,2,1,1,1,1,2,2
2,1,2,2,1,2,1,1,1,1,1,1
2,2,1,1,2,2,1,1,1,1,2,1


In [44]:
print(dim(geno.glm))
head(glm.data, 2)

[1] 11170    12


,geno.glm,sex,pd_case
,"<dbl[,12]>",<int>,<dbl>
1,"2, 2, 1, 2, 2, 2, 1, 1, 1, 1, 2, 2",1,1
2,"2, 1, 2, 2, 1, 2, 1, 1, 1, 1, 1, 1",2,1


### i) Haplotype analysis: _PD ~ sex + haplotype + 

In [45]:
# Haplotype glm fit plux sex as covariate,
# return model matrix
#fm <- glm(case ~ age + sex + hap1, data = data, family =”binomial”)
fit.pd <- haplo.glm(pd_case ~ sex + geno.glm, family = binomial, data=glm.data, na.action = "na.geno.keep",
                     locus.label=snp_names)

In [46]:
summary(fit.pd)


Call:
haplo.glm(formula = pd_case ~ sex + geno.glm, family = binomial, 
    data = glm.data, na.action = "na.geno.keep", locus.label = snp_names)

Coefficients:
                   coef        se    t.stat  pval
(Intercept)    0.030480  0.088122  0.345888 0.729
sex           -0.176293  0.041108 -4.288481 0.000
geno.glm.1    -0.047046  0.120463 -0.390546 0.696
geno.glm.5    -0.015278  0.098667 -0.154845 0.877
geno.glm.6    -0.075741  0.081269 -0.931977 0.351
geno.glm.8     0.166059  0.079485  2.089189 0.037
geno.glm.10    0.004615  0.151635  0.030436 0.976
geno.glm.12    0.073384  0.142212  0.516014 0.606
geno.glm.13    0.004109  0.073330  0.056034 0.955
geno.glm.14   -0.084145  0.108076 -0.778569 0.436
geno.glm.15   -0.316853  0.071988 -4.401482 0.000
geno.glm.17    0.295785  0.143775  2.057276 0.040
geno.glm.18   -0.104756  0.100594 -1.041377 0.298
geno.glm.22   -0.074021  0.057716 -1.282517 0.200
geno.glm.23   -0.056528  0.069936 -0.808281 0.419
geno.glm.26    0.097263  0.089200  1.0

In [437]:
#help(haplo.glm)

In [425]:
cat('base habplotype index: ', fit.pd$haplo.base, '\n')

base habplotype index:  9 


In [429]:
cat('Number of common haplotypes',length(fit.pd$haplo.common), '\n')
cat('Common haplotypes', fit.pd$haplo.common)

Number of common haplotypes 16 
Common haplotypes 1 5 6 8 10 12 13 14 15 17 18 22 23 26 30 31

In [503]:
length(fit.pd$coefficients)

[1] 19

### Get Haplotype Frequencies for Cases and Controls and Compute Confidence Intervals

In [48]:
summary_fit=(summary(fit.pd))

In [51]:
print(dim(fit.pd$haplo.unique))
head(fit.pd$haplo.unique, 3)

[1] 33  6


,rs1467967,rs242557,rs3785883,rs2471738,rs8070723,rs7521
1,A,A,A,C,A,A
2,A,A,A,C,A,G
3,A,A,A,T,A,A


In [54]:
# Merge UNIQUE Haplotypes with Haplotype Frequencies
haplo_unique <- fit.pd$haplo.unique   # Matrix of unique haplotypes
haplo_freq <- fit.pd$haplo.freq       # Numeric vector of haplotype frequencies

# Convert haplo_unique to a data frame and assign SNP names as column names
haplo_unique_df <- as.data.frame(haplo_unique, stringsAsFactors = FALSE)
colnames(haplo_unique_df) <- colnames(haplo_unique)

# Add haplotype frequencies as a new column
haplo_unique_df$frequency <- haplo_freq

# View Final data frame
#head(haplo_unique_df,3)
print(dim(haplo_unique_df))

## Add Haplotype frequencies for cases and controls
merged_haplo_data= merge(haplo_unique_df, freq_haplotypes, by= colnames(haplo_unique), all.x = TRUE)
cat('Total number of  unique haplotypes: ', dim(merged_haplo_data)[1], '\n')

merged_haplo_data <- merged_haplo_data %>%
  rename(freq_controls = `phenotype=0`, freq_cases = `phenotype=1`)
merged_haplo_data <- merged_haplo_data %>%
    mutate(haplo_index=rownames(merged_haplo_data))

### Compute Confidence Intervals
## First format summary stats
sum_stats <- summary_fit$coefficients
sum_stats <- data.frame(
  Variable = rownames(sum_stats), # Add row names as a new column called "Variable"
  sum_stats,                      # Add the rest of the data
  row.names = NULL                # Remove the row names
)

### Calculate the Odds Ratio and Confidence Intervals
# -Thecoef values represent log-odds (logarithm of the odds ratio) rather than the odds ratios themselves.
# -To interpret them as odds ratios, you need to exponentiate the coefficients.
#- Converting Coefficients to Odds Ratios
sum_stats <- sum_stats %>%
  mutate(
    OR = exp(coef),                            # Odds Ratio
    CI_lower = exp(coef - 1.96 * se),           # Lower 95% CI
    CI_upper = exp(coef + 1.96 * se),            # Upper 95% CI
  )

### Merge the haplotype frequency data with summary stats
## Firts split the variacle column
# Transforming the data
sum_stats <- sum_stats %>%
  mutate(
    haplo_index = ifelse(
      Variable == "(Intercept)", "1001",
      ifelse(Variable == "sex", "1002",
      ifelse(Variable == "geno.glm.rare", "1003",
             as.character(sub(".*\\.(\\d+)$", "\\1", Variable))
      )
    )
  )
)
## merge Haplotype  Frequencies and summary stats
merged_haplo_stats= merge(merged_haplo_data, sum_stats, by='haplo_index', ) 
merged_haplo_stats$OR_95_CI <- paste0(round(merged_haplo_stats$OR,2), " (", round(merged_haplo_stats$CI_lower, 2), "-", round(merged_haplo_stats$CI_upper, 2), ")")

## Apply bonferroni correction to pvalues
ntests= nrow(merged_haplo_stats) # Number of tests (Only contains the cmmon haplotypes)
# Apply Bonferroni correction
merged_haplo_stats$pval_corrected <- pmin(merged_haplo_stats$pval * ntests, 1)
Bonferoni_pval=0.05/ntests
cat('Number od haplotypes with frequency >= 1%: ', ntests, '\n')
cat('Number to association test performed: ', ntests, '\n')
cat('Bonferroni corrected pvalue Threshold: ', round(Bonferoni_pval, 4),  '\n')
cat('Number of haplotypes significatnt at p < 0.05: ' , dim(merged_haplo_stats[merged_haplo_stats$pval < 0.05])[2],  '\n')
cat('Number of haplotypes significatnt at p < ', Bonferoni_pval, ': ', dim(merged_haplo_stats[merged_haplo_stats$pval < Bonferoni_pval])[2],  '\n')

##BASE HAPLOTYPE
# ===== haplo.base            A        G         A         C         A      A  0.17716 ===========

[1] 33  7
Total number of  unique haplotypes:  33 
Number od haplotypes with frequency >= 1%:  16 
Number to association test performed:  16 
Bonferroni corrected pvalue Threshold:  0.0031 
Number of haplotypes significatnt at p < 0.05:  4 
Number of haplotypes significatnt at p <  0.003125 :  1 


In [55]:
cat('Number of haplotypes significatnt at p < 0.05: ' , dim(merged_haplo_stats[merged_haplo_stats$pval < 0.05])[2],  '\n')
merged_haplo_stats[merged_haplo_stats$pval < 0.05,]

Number of haplotypes significatnt at p < 0.05:  4 


,haplo_index,rs1467967,rs242557,rs3785883,rs2471738,rs8070723,rs7521,frequency,Total,freq_controls,⋯,Variable,coef,se,t.stat,pval,OR,CI_lower,CI_upper,OR_95_CI,pval_corrected
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,⋯,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>
6,15,A,G,G,C,G,G,0.04950761,0.04950761,0.05531113,⋯,geno.glm.15,-0.3168534,0.07198788,-4.401482,1.085139e-05,0.7284376,0.6325794,0.8388217,0.73 (0.63-0.84),0.0001736223
7,17,A,G,G,T,A,G,0.01705034,0.01699720,0.01459393,⋯,geno.glm.17,0.2957846,0.14377489,2.057276,3.968291e-02,1.3441806,1.0140841,1.7817274,1.34 (1.01-1.78),0.6349265512
12,30,G,G,G,C,A,A,0.12971287,0.12971663,0.13480915,⋯,geno.glm.30,-0.1242714,0.05651571,-2.198882,2.790680e-02,0.8831402,0.7905375,0.9865902,0.88 (0.79-0.99),0.4465088329
16,8,A,A,G,T,A,G,0.04604574,0.04603415,0.04158320,⋯,geno.glm.8,0.1660592,0.07948503,2.089189,3.671332e-02,1.1806430,1.0103215,1.3796777,1.18 (1.01-1.38),0.5874131664


In [56]:
cat('Number of haplotypes significatnt at p < ', Bonferoni_pval, ': ', dim(merged_haplo_stats[merged_haplo_stats$pval < Bonferoni_pval])[2],  '\n')
merged_haplo_stats[merged_haplo_stats$pval < Bonferoni_pval, ]


Number of haplotypes significatnt at p <  0.003125 :  1 


,haplo_index,rs1467967,rs242557,rs3785883,rs2471738,rs8070723,rs7521,frequency,Total,freq_controls,⋯,Variable,coef,se,t.stat,pval,OR,CI_lower,CI_upper,OR_95_CI,pval_corrected
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,⋯,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>
6,15,A,G,G,C,G,G,0.04950761,0.04950761,0.05531113,⋯,geno.glm.15,-0.3168534,0.07198788,-4.401482,1.085139e-05,0.7284376,0.6325794,0.8388217,0.73 (0.63-0.84),0.0001736223


In [57]:
# Create 'tag_h1' and 'tag_h2' columns based on 'rs8070723' values
# Step 1: Create tag_h1, tag_h2, and tag_h1c_temp
merged_haplo_stats <- merged_haplo_stats %>%
  mutate(
    tag_h1 = ifelse(rs8070723 == "A", "H1", NA),
    tag_h2 = ifelse(rs8070723 == "G", "H2", NA),
    tag_h1c_temp = ifelse(rs1467967 == "A" & rs242557 == "A", "H1c", NA)
  )

# Step 2: Assign H1c to the row with the maximum freq_cases among those where tag_h1c_temp is H1c
merged_haplo_stats <- merged_haplo_stats %>%
  mutate(
    tag_h1c = ifelse(tag_h1c_temp == "H1c" & 
                     freq_cases == max(freq_cases[tag_h1c_temp == "H1c"], na.rm = TRUE), "H1c", NA)
  ) %>%
  select(-tag_h1c_temp) # Drop the temporary column

# Define the labels for tag_h1sub
labels_h1sub <- c('H1b', 'H1d', 'H1e', 'H1f', 'H1g', 'H1i', 'H1j', 'H1k', 'H1m', 'H1n', 'H1o', 'H1p', 'H1q', 'H1r', 'H1x', 'H1y', 'H1z')

# Create the 'tag_h1sub' column based on specified conditions
merged_haplo_stats <- merged_haplo_stats %>%
  # Filter for relevant rows and sort them
  arrange(desc(freq_cases)) %>%
  mutate(tag_h1sub = case_when(
    !is.na(tag_h1) & is.na(tag_h2) & is.na(tag_h1c) ~ labels_h1sub[row_number()],
    TRUE ~ NA_character_
  ))

# Create the 'haplotype_tag' column based on the first non-NA value from tag_h2, tag_h1c, and tag_h1sub
merged_haplo_stats <- merged_haplo_stats %>%
  mutate(haplotype_tag = coalesce(tag_h2, tag_h1c, tag_h1sub))

# Sort the data frame by haplotype_tag
merged_haplo_stats <- merged_haplo_stats %>%
  arrange(haplotype_tag)

### Save MAPT Happlotype analysis
# Save the data frame as a tab-delimited file
write.table(merged_haplo_stats, file = "table_MAPT_haplotype_association_with_pd_case_control.tsv", sep = "\t", row.names = FALSE, quote = TRUE)
## Save selected columns for paper

#### Save Haplotype Analysis Table
* Table: Association between MAPT haplotype and PD risk

In [ ]:
# Select the specified columns
output_mapt_analysis <- merged_haplo_stats %>%
  select(
    haplotype_tag,
    rs1467967,
    rs242557,
    rs3785883,
    rs2471738,
    rs8070723,
    rs7521,
    freq_cases,
    freq_controls,
    OR_95_CI,
    pval,
    haplo_index,
    pval_corrected,
    frequency,
    se,
    coef,
    CI_lower,
    CI_upper
  )

# Save the output as a tab-delimited file
write.table(output_mapt_analysis, file = "table_MAPT_haplotype_association_with_pd_case_control_paper.tsv", sep = "\t", row.names = FALSE, quote = TRUE)